# Préambule

Paul et Béatrice ont préparé les données des exports Toflit18 pour les provinces + dans carte matricielle voir quel port exporte quoi et à quel niveau (+ même chose en import).

Objectif : transposer ces données en carte au niveau province.

- par province, avoir le camembert des imports et exports
- faire sauter aux yeux les provinces exportatrices et les provinces exportatrices

In [10]:
# Load an empty map
from keplergl import KeplerGl
from poitousprint import Portic, get_online_csv
import json
import pandas as pd
import geopandas

In [30]:
# step 1 : récupérer les données

mappings_list = get_online_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSDyf3-ZL7AjZTal84kEwXThfchUW8K9XzEBSzex0H6RzuQkB5u19x9xnHhEBmcSLZBGCk5EY5gJji-/pub?output=csv")

mappings = {}
for m in mappings_list:
    mappings[m['customs_region']] = m['province']

In [17]:
# step 2 : préparation pour la visualisation
imports = pd.read_csv("imports.csv")
exports = pd.read_csv("exports.csv")

def clean_and_join (df):
    df["source"] = df.source.str[3:]
    df["target"] = df.target.str[3:]
    df['source_province'] = df.source.map(mappings)
    return df

def sum_by_province (df) :
    return df.groupby('source_province')['value'].sum()
    
imports = clean_and_join(imports)
exports = clean_and_join(exports)

imports.source_province.value_counts()

exports_summed = sum_by_province(exports)
imports_summed = sum_by_province(imports)

exports_summed.to_csv('exports_summed.csv')
imports_summed.to_csv('imports_summed.csv')

In [27]:
# step 3 : charger le fond de carte et faire la jointure avec les données d'imports et d'exports

polygons = geopandas.read_file("cartoweb_france_1789_geojson.geojson")
polygons["import_values"] = polygons.shortname.map(imports_summed).fillna(0)
polygons["export_values"] = polygons.shortname.map(exports_summed).fillna(0)

In [29]:
# step 4 : visualisation

config = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

"""
# create data frame
df_with_geometry = pd.DataFrame({
    'id': [1],
    'geometry_string': [json.dumps(background)]
})

# add to map
map_1.add_data(df_with_geometry, "df_with_geometry")
"""


map = KeplerGl(config=config )
map.add_data(data=polygons, name="feature_collection")

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'mapState': {'latitude': 45.6876849, 'longitude': -1.15, 'zoom': …